# Data Exploration

In [ ]:
# Loading data
xtest = pd.read_csv('xtest_challenge.csv')

# Feature Engineering

# Algorithms

## Run a randomized searchCV with XGBoostClassifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Classifier
xgb = XGBClassifier(
    n_jobs=-1,
    silent=False)

# Create hyperparameter options
xgb_max_depth=[3, 5, 7, 10]             # Usual values between 3-10
xgb_learning_rate=[0.1, 0.5, 1, 1.2]    # Makes the model more robust by shrinking the weights on each step
xgb_n_estimators=[100, 200, 500, 1000, 1200]
xgb_booster=['gbtree']                  #, 'gblinear', 'dart']
xgb_reg_lambda=[1, 2]                   # L2 used to reduce overfitting

hyperparameters = dict(
    max_depth = xgb_max_depth, 
    learning_rate = xgb_learning_rate,
    n_estimators = xgb_n_estimators,
    booster=xgb_booster,
    reg_lambda=xgb_lambda)

# Create randomized grid search
rscv = RandomizedSearchCV(xgb, hyperparameters, random_state=1, n_iter=50, cv=5, verbose=10, n_jobs=-1)
# Fit randomized search
best_model = rscv.fit(xtrain_part, ytrain_part)

# View Hyperparameter Values Of Best Model
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])

## Save a trained model
Credit : https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [ ]:
import pickle

model = LogisticRegression()
model.fit(X_train, Y_train)

# Save the model to disk
lrmodel_filename = 'saved_lr.sav'
pickle.dump(model, open(lrmodel_filename, 'wb'))
 
# Load the model from disk
loaded_model = pickle.load(open(lrmodel_filename, 'rb'))
result = loaded_model.score(X_test, Y_test)

# Challenges

## Making a submission

In [4]:
def make_submission(test_data, algorithm, filename='submission.csv'):
    """Creates a CSV file for challenge submission
  
    test_data: Description of arg1 
    algorithm: Algo used for making prediction
    filename: 'submission.csv'
    """
    ytest = algorithm.predict(test_data)
    np.savetxt(fichier, ytest, fmt = '%1.0d', delimiter=',')